In [4]:
import scipy         
from scipy.linalg import expm
import scipy.sparse as sparse
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [14]:
L = 7 #system size
M = 0.1 #fermion mass
N=4 #number of layers
l=1 #spin length
m_epsilon = 0
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [15]:
#construct basis
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
        0.         |2 2 2 2 2 2 2>           2186  
        1.         |2 2 2 2 2 2 1>           2185  
        2.         |2 2 2 2 2 2 0>           2184  
        3.         |2 2 2 2 2 1 2>           2183  
        4.         |2 2 2 2 2 1 1>           2182  
        5.         |2 2 2 2 2 1 0>           2181  
        6.         |2 2 2 2 2 0 2>           2180  
        7.         |2 2 2 2 2 0 1>           2179  
        8.         |2 2 2 2 2 0 0>           2178  
        9.         |2 2 2 2 1 2 2>           2177  
       10.         |2 2 2 2 1 2 1>           2176  
       11.         |2 2 2 2 1 2 0>           2175  
       12.         |2 2 2 2 1 1 2>           2174  
       13.         |2 2 2 2 1 1 1>           2173  
       14.         |2 2 2 2 1 1 0>           2172  
       15.         |2 2 2 2 1 0 2>           2171  
       16.         |2 2 2 2 1 0 1>           2170  
       17.         |2 2 2 2 1 0 0>           21

In [16]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []
gauss_law_matrix = []
z_comp_minf = [[-m_epsilon,0]]
z_comp_pinf = [[m_epsilon,L-1]]

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z",liste[0]],
            ["I",z_comp_minf],

        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",z_comp_pinf],
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",liste[i-1]],
            ["I",z_comp_pinf],

        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],  
            ["I",liste[i]]
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))
for i in range(L+1):
    gauss_law_matrix.append(Gauss_law[i].tocsc())

In [17]:
#projectors on g=1 for every site
Proj = [sparse.csr_matrix((basis.Ns,basis.Ns)) for i in range(L+1)]
for i in range(L+1):
    for j in range(basis.Ns):
        if (np.real(gauss_law_matrix[i][j,j])>=1.-1e-8) & (np.real(gauss_law_matrix[i][j,j])<=1.+1e-8):
            Proj[i][j,j] = 1.

In [18]:
A = np.zeros((2*l+1,2*l+1))
for i in range(2*l+1):
    for j in range(2*l+1):
        A[i][j] = (-l+i)**j
b = np.zeros(2*l+1)
for i in range(2*l+1):
    b[i] = (-1)**(-l+i)
solution = scipy.linalg.solve(A,b)
x = np.zeros(2*3+1)
for i in range(2*l+1):
    x[i] = solution[i]
print(x)

[ 1.  0. -2.  0.  0.  0.  0.]


In [19]:
const_term = []
linear_term = []
quadratic_term = []
cubic_term = []
quartic_term = []
quintic_term = []
sextic_term = []
interaction_p = []
interaction_m = []

for i in range(L-1):
    const_term.append([[x[0]/(2*np.sqrt(l*(l+1))),i]])
    linear_term.append([[x[1]/(2*np.sqrt(l*(l+1))),i,i+1]])
    quadratic_term.append([[x[2]/(2*np.sqrt(l*(l+1))),i,i+1,i+1]])
    cubic_term.append([[((-1)**i*x[3])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1]])
    quartic_term.append([[((-1)**i*x[4])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1]])
    quintic_term.append([[((-1)**i*x[5])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1,i+1]])
    sextic_term.append([[((-1)**i*x[6])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1,i+1,i+1]])

    const_term

    interaction_p.append([
        ["+", const_term[i]],
        ["+z", linear_term[i]],
        ["+zz", quadratic_term[i]],
        ["+zzz", cubic_term[i]],
        ["+zzzz", quartic_term[i]],
        ["+zzzzz", quintic_term[i]],
        ["+zzzzzz", sextic_term[i]],

    ])

    interaction_m.append([
        ["-", const_term[i]],
        ["-z", linear_term[i]],
        ["-zz", quadratic_term[i]],
        ["-zzz", cubic_term[i]],
        ["-zzzz", quartic_term[i]],
        ["-zzzzz", quintic_term[i]],
        ["-zzzzzz", sextic_term[i]],

    ])

const_term.append([[(0.5*(-1)**m_epsilon)/np.sqrt(l*(l+1)),L-1]])

interaction_p.append([
    ["+", const_term[L-1]]
])
interaction_m.append([
    ["-", const_term[L-1]]
])

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[2*(-1)**i*M,i] for i in range(L)]

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

H_int_p = []
H_int_m = []
interaction = [sparse.csr_matrix((basis.Ns,basis.Ns)) for i in range(L)]
h_int = sparse.csr_matrix((basis.Ns,basis.Ns))
for i in range(L):
    H_int_p.append(hamiltonian(interaction_p[i],dynamic_list=[],basis=basis,**no_checks))
    H_int_m.append(hamiltonian(interaction_m[i],dynamic_list=[],basis=basis,**no_checks))
    interaction[i] = (Proj[i]@H_int_p[i].tocsc()@Proj[i+1]+Proj[i+1]@H_int_m[i].tocsc()@Proj[i])

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)
h_kin_mass = H_kin_mass.tocsc()

h_int = 0

for i in range(L):
    h_int += interaction[i]


full_ham_matrix = h_kin_mass + h_int #Spin-1 Hamiltonian without the local contstaints

G=0
for i in range(L+1):
    G += 1e2*(Gauss_law[i]**2-Gauss_law[i])**2
constraint_full_matrix = G.tocsc()
constrained_full_ham_matrix= full_ham_matrix + constraint_full_matrix #Spin-1 Hamiltonian with the local contstaints as a penalty term
#print(full_ham_matrix)

In [20]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian without the local contstaints
eigenval, eigenvec = scipy.sparse.linalg.eigsh(full_ham_matrix,k = 3,which = "SA")
print(eigenval[eigenval<=1e1])
print(eigenvec[:,0])

[-1.43920377 -0.7416023  -0.7416023 ]
[ 1.67222932e-04-3.99252790e-04j  8.27369888e-19-1.78963353e-18j
 -9.01286370e-18-3.08409599e-18j ...  1.07763301e-17-4.05597766e-18j
  3.56354011e-18-1.12828039e-18j  2.45971725e-18+4.06685870e-18j]


In [21]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian with the local contstaints
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(constrained_full_ham_matrix,k = 3,which = "SA")


In [22]:
#Gauge invariant part of the spectrum
string = ""
print(eigenvalues[eigenvalues<=1e1])
for i in range(basis.Ns):
    if np.abs(eigenvectors[:,0][i])>=1e-5:
        string+=str(eigenvectors[:,0][i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)


[-1.43920377 -0.43399733 -0.32159336]
(-0.0001985761560061991-0.0003846216450841934j)|2 2 2 2 2 2 2>
(-0.0007260758863404419-0.0014063345147303434j)|2 2 2 2 2 1 2>
(0.001350100443647668+0.002615006073235662j)|2 2 2 2 2 1 1>
(-0.0005888948857782878-0.0011406289880726561j)|2 2 2 1 2 2 2>
(-0.0027504942739503056-0.005327425277734931j)|2 2 2 1 2 1 2>
(0.005344237249706078+0.010351239369798407j)|2 2 2 1 2 1 1>
(0.005209099050309755+0.010089490539391663j)|2 2 2 1 1 1 2>
(-0.01137753116172624-0.022037110815104764j)|2 2 2 1 1 1 1>
(0.004266193861392585+0.008263179906393624j)|2 2 2 1 1 0 1>
(-0.0007260758864272338-0.0014063345147618785j)|2 1 2 2 2 2 2>
(-0.002728087460729509-0.0052840255785972234j)|2 1 2 2 2 1 2>
(0.0051063685583517355+0.009890512113557782j)|2 1 2 2 2 1 1>
(-0.002750494273921338-0.005327425277723116j)|2 1 2 1 2 2 2>
(-0.01310180088706701-0.025376844406765537j)|2 1 2 1 2 1 2>
(0.02555710572355814+0.04950149227776342j)|2 1 2 1 2 1 1>
(0.02586955999797733+0.05010668415739693j)|2 1

In [11]:
#initial state
string = ""
for i in range(L):
    string +="1"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.

In [12]:
#cost function of the initial state
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

0.0


In [13]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

0j


In [14]:
def rot_sigma(k,i,j,phi,theta):
    if k>=L:
        print("Warning! Rotations apply to k<=L-1!")
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    rot_matrix = expm(-1j*theta*sigma/2)
    if k==0:
        tensprod = rot_matrix
        for i in range(L-1):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    else:
        tensprod = sparse.identity(2*l+1)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+1,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    return tensprod

def rot_ms(k,i,j,phi,theta):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    power_matrix = scipy.sparse.kron(sigma,sparse.identity(2*l+1))+scipy.sparse.kron(sparse.identity(2*l+1),sigma)
    rot_matrix = scipy.sparse.linalg.expm(-1j*power_matrix@power_matrix*theta/4)
    if k == 0:
        tensprod = rot_matrix
        for i in range(L-2):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    else:
        tensprod = sparse.identity(2*l+1)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+2,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    return tensprod

def rot_ms_e(i,j,phi,theta):
    ms_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        ms_e = rot_ms(2*k,i,j,phi,theta)@ms_e
    return(ms_e)

def rot_ms_o(i,j,phi,theta):
    ms_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)-1):
        ms_o = rot_ms(2*k+1,i,j,phi,theta)@ms_o
    return(ms_o)

def rot_sigma_e(i,j,phi,theta):
    sigma_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        sigma_e = rot_sigma(2*k,i,j,phi,theta)@sigma_e
    return(sigma_e)

def rot_sigma_o(i,j,phi,theta):
    sigma_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)):
        sigma_o = rot_sigma(2*k+1,i,j,phi,theta)@sigma_o
    return(sigma_o)

def rot_sigma_all(i,j,phi,theta):
    sigma_all = sparse.identity(basis.Ns,dtype = complex)
    for k in range(L):
        sigma_all = rot_sigma(k,i,j,phi,theta)@sigma_all
    return(sigma_all)

In [15]:
def ms_hamiltonian(i,j,phi):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))

    sigma = sparse.csc_matrix(sigma)
    summe = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for k in range(L):
        if k == 0:
            tensprod = sigma
            for m in range(L-1):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = tensprod.toarray()
        else:
            tensprod = sparse.identity(2*l+1)
            for i in range(1,k):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = scipy.sparse.kron(tensprod,sigma)
            for i in range(k+1,L):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = tensprod.toarray()
        summe +=tensprod
    return summe@summe
                
def sigma_z_part(i,j):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-j][2*l-j] = -1.
    sigma = sparse.csc_matrix(sigma)
    summe = 0
    for k in range(L):
        tensprod = 0
        if k==0:
            tensprod = sigma
            for i in range(L-1):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        else:
            tensprod = sparse.identity(2*l+1)
            for i in range(1,k):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = scipy.sparse.kron(tensprod,sigma)
            for i in range(k+1,L):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        summe +=tensprod
    return summe
    
matrix_part = ms_hamiltonian(1,2,0)
ms_ham_eigenval, ms_ham_eigenvec = scipy.linalg.eigh(matrix_part)




def ms_gate_global(theta):
    exponential = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(basis.Ns):
        exponential[i][i] = np.exp(1j*ms_ham_eigenval[i]*theta/4)
    exponential = scipy.sparse.csc_matrix(exponential)
    return ms_ham_eigenvec@exponential@ms_ham_eigenvec.T.conj()

    

In [16]:
#import timeit
#print(timeit.timeit('[ms_gate_global(0,2,np.pi)]', globals=globals()))
import datetime
print(datetime.datetime.now())
a = ms_gate_global(np.pi)
print(datetime.datetime.now())
print(np.size(a))


2022-03-23 09:01:15.955316
2022-03-23 09:01:16.656929
4782969


In [17]:
def cost_function_sigma(theta):
    psi_var = psi_0
    for i in range(N):
        psi_var = ms_gate_global(theta[5*i+0])@psi_var
        psi_var = rot_sigma_e(0,2,0,theta[5*i+2])@rot_sigma_e(0,1,0,theta[5*i+1])@psi_var
        psi_var = rot_sigma_o(0,2,0,theta[5*i+4])@rot_sigma_o(0,1,0,theta[5*i+3])@psi_var
        psi_var = rot_sigma(0,0,2,0,theta[5*i+2])@rot_sigma(0,0,1,0,theta[5*i+1])@psi_var
        psi_var = rot_sigma(L-1,0,2,0,theta[5*i+2])@rot_sigma(L-1,0,1,0,theta[5*i+1])@psi_var
        
    cost = np.conj(psi_var)@full_ham_matrix@psi_var
    return np.real(cost)

import datetime

print(datetime.datetime.now())
a = cost_function_sigma(np.pi*np.ones(5*N))
print(datetime.datetime.now())

2022-03-23 09:01:16.662684
2022-03-23 09:01:19.336270


In [18]:
opt_params = []
function_values = []
def callback_function(x,fun,context):
    opt_params.append(x)
    function_values.append(fun)


In [19]:
full_ham_matrix = sparse.csc_matrix(full_ham_matrix)

In [20]:
theta = np.zeros(5*N)
import optuna
#cost function definition
def cost_function_sigma(trial):
    psi_var = psi_0
    for i in range(N):
        theta[5*i+0] = trial.suggest_float("5*"+str(i)+"+0", -3*np.pi, 3*np.pi)
        theta[5*i+1] = trial.suggest_float("5*"+str(i)+"+1", -3*np.pi, 3*np.pi)
        theta[5*i+2] = trial.suggest_float("5*"+str(i)+"+2", -3*np.pi, 3*np.pi)
        theta[5*i+3] = trial.suggest_float("5*"+str(i)+"+3", -3*np.pi, 3*np.pi)
        theta[5*i+4] = trial.suggest_float("5*"+str(i)+"+4", -3*np.pi, 3*np.pi)


        psi_var = ms_gate_global(theta[5*i+0])@psi_var
        psi_var = rot_sigma_e(0,2,0,theta[5*i+2])@rot_sigma_e(0,1,0,theta[5*i+1])@psi_var
        psi_var = rot_sigma_o(0,2,0,theta[5*i+4])@rot_sigma_o(0,1,0,theta[5*i+3])@psi_var
        psi_var = rot_sigma(0,0,2,0,theta[5*i+2])@rot_sigma(0,0,1,0,theta[5*i+1])@psi_var
        psi_var = rot_sigma(L-1,0,2,0,theta[5*i+2])@rot_sigma(L-1,0,1,0,theta[5*i+1])@psi_var
        
    cost = np.conj(psi_var)@full_ham_matrix@psi_var
    return np.real(cost)

if __name__ == "__main__":
    sampler = optuna.samplers.CmaEsSampler()
    study = optuna.create_study(sampler=sampler)
    study.optimize(cost_function_sigma, n_trials=3000)

[I 2022-03-23 09:01:19,935] A new study created in memory with name: no-name-9ab81b13-91ba-4aaa-8b65-0433a4414084
[I 2022-03-23 09:01:22,612] Trial 0 finished with value: 2.4932316305185496 and parameters: {'5*0+0': -2.2791306385131325, '5*0+1': -2.0345196110208574, '5*0+2': 4.462979752194791, '5*0+3': 1.651237060023897, '5*0+4': 8.440635845840294, '5*1+0': -6.360966682253039, '5*1+1': 0.5513301824333254, '5*1+2': -6.369915264046634, '5*1+3': 8.699132436544247, '5*1+4': -9.306386835719211, '5*2+0': 2.7162428387114197, '5*2+1': 7.483425886630684, '5*2+2': -1.2238106175227745, '5*2+3': -5.531162679117468, '5*2+4': 3.4895913641077776, '5*3+0': 2.2966006475472955, '5*3+1': -0.5126406194675788, '5*3+2': -0.3037214674606066, '5*3+3': -3.631369300921386, '5*3+4': 1.2827624508345732}. Best is trial 0 with value: 2.4932316305185496.
[I 2022-03-23 09:01:25,130] Trial 1 finished with value: 2.5067818820782968 and parameters: {'5*0+0': 2.6634327389573174, '5*0+1': -1.8328561039871845, '5*0+2': -1.

[I 2022-03-23 09:01:50,560] Trial 11 finished with value: 1.8118501470096238 and parameters: {'5*0+0': -1.0248870949965485, '5*0+1': -1.1016591828856863, '5*0+2': 3.2557835279428913, '5*0+3': 0.14868854803530787, '5*0+4': -2.1024695848522037, '5*1+0': 5.608214809444429, '5*1+1': -2.3022196413890788, '5*1+2': -1.1795602698328804, '5*1+3': 1.1019357874341134, '5*1+4': -2.8017644573804197, '5*2+0': -1.0581063672390527, '5*2+1': 0.13758214007695455, '5*2+2': 0.7468939241253324, '5*2+3': -1.3647324027350645, '5*2+4': 0.9471836127869316, '5*3+0': 0.5055030388315805, '5*3+1': 0.092189899852812, '5*3+2': -3.4560438424622606, '5*3+3': 1.2752430017498102, '5*3+4': -3.1050672421957297}. Best is trial 11 with value: 1.8118501470096238.
[I 2022-03-23 09:01:53,090] Trial 12 finished with value: 2.8252014480074195 and parameters: {'5*0+0': -0.11641213845142137, '5*0+1': 3.354203271675265, '5*0+2': 2.8675036914437535, '5*0+3': -0.7056950875580856, '5*0+4': 1.0852442368803772, '5*1+0': 1.01510183689337

[I 2022-03-23 09:02:20,588] Trial 23 finished with value: 2.295245023986676 and parameters: {'5*0+0': 3.599156546096307, '5*0+1': 3.9485058190900117, '5*0+2': 3.564999792824283, '5*0+3': -0.5240482292409394, '5*0+4': 0.9033831309244208, '5*1+0': 2.7339334616172923, '5*1+1': 2.0751499340168644, '5*1+2': -5.765643726452018, '5*1+3': 8.0980198788448, '5*1+4': 1.8008178074704548, '5*2+0': 0.7494441629443532, '5*2+1': 1.1158478266870833, '5*2+2': 0.9988532455354514, '5*2+3': -1.1829233873063028, '5*2+4': 3.524695827392767, '5*3+0': 0.015218238557124453, '5*3+1': 3.4149557852278067, '5*3+2': -4.308314606238075, '5*3+3': -2.9536502279100643, '5*3+4': -2.6002544058270796}. Best is trial 14 with value: 1.4584049305444378.
[I 2022-03-23 09:02:23,107] Trial 24 finished with value: 2.0472617789464653 and parameters: {'5*0+0': 0.6658590368025151, '5*0+1': 2.4081987219237657, '5*0+2': -2.9571973849554647, '5*0+3': 0.9522598293037409, '5*0+4': 5.8156034462721, '5*1+0': 3.9234357779800595, '5*1+1': 3.

[I 2022-03-23 09:02:50,782] Trial 35 finished with value: 2.9675018910029127 and parameters: {'5*0+0': 1.7673749264177039, '5*0+1': -3.3654150193657264, '5*0+2': -0.6160846538300004, '5*0+3': 9.424297423975485, '5*0+4': -0.3110224566159048, '5*1+0': 4.754015867270868, '5*1+1': 0.565077618119824, '5*1+2': 5.4255862738654885, '5*1+3': -2.6775844003192657, '5*1+4': -0.3344315563672845, '5*2+0': 0.08227990156350284, '5*2+1': 1.6006489035479117, '5*2+2': -3.0124681375882894, '5*2+3': -4.357103895662101, '5*2+4': -0.10893641902743278, '5*3+0': 1.1721445203086982, '5*3+1': -3.119484084826985, '5*3+2': -7.549671161130222, '5*3+3': 5.503079630237206, '5*3+4': -6.844851690924742}. Best is trial 14 with value: 1.4584049305444378.
[I 2022-03-23 09:02:53,309] Trial 36 finished with value: 2.5206772824676715 and parameters: {'5*0+0': -3.8768106660753645, '5*0+1': -3.6003771412400862, '5*0+2': 2.8722220830368688, '5*0+3': 3.5509421247222774, '5*0+4': -0.930238047344468, '5*1+0': 3.6887109739893162, '

[I 2022-03-23 09:03:20,918] Trial 47 finished with value: 1.3897265529769065 and parameters: {'5*0+0': -1.4741908551505551, '5*0+1': -2.592626228669393, '5*0+2': 1.7730831469743824, '5*0+3': 3.1893556736993243, '5*0+4': 4.287680866119173, '5*1+0': 0.9923020375944271, '5*1+1': -0.9501360403618886, '5*1+2': 1.1513094786081137, '5*1+3': -1.3963187515400013, '5*1+4': 2.571477910088497, '5*2+0': 3.8226192848348868, '5*2+1': 3.4592879965153425, '5*2+2': -3.5497925103039325, '5*2+3': -1.7741085865909898, '5*2+4': 4.1904688802681545, '5*3+0': 2.2225383966766032, '5*3+1': -0.7196070565019712, '5*3+2': -7.970535568769652, '5*3+3': 5.7889468007095255, '5*3+4': -1.2164034630722251}. Best is trial 47 with value: 1.3897265529769065.
[I 2022-03-23 09:03:23,378] Trial 48 finished with value: 2.6679088490501934 and parameters: {'5*0+0': 0.5334816178603262, '5*0+1': -2.609570630700491, '5*0+2': -1.1315108536885758, '5*0+3': 1.5724446209788887, '5*0+4': 1.764984743694971, '5*1+0': 1.485832760463592, '5*1

...

[I 2022-03-23 11:28:52,248] Trial 2951 finished with value: -1.288411906956877 and parameters: {'5*0+0': -0.11503410285903799, '5*0+1': 7.1502813137409875, '5*0+2': 0.38802848722400385, '5*0+3': -4.851238925753708, '5*0+4': 2.454435219835315, '5*1+0': 0.15827447933709496, '5*1+1': 8.781851255772965, '5*1+2': 6.767475211319561, '5*1+3': -5.297512545449691, '5*1+4': 4.407342709471437, '5*2+0': 5.965897798163292, '5*2+1': 3.4011066543870467, '5*2+2': 7.4251443724828725, '5*2+3': 0.6530032033518526, '5*2+4': -5.675717513332789, '5*3+0': -0.10036868122463644, '5*3+1': 4.735755799192517, '5*3+2': 1.631973737650578, '5*3+3': -3.0964362105164214, '5*3+4': 4.70720756939668}. Best is trial 2951 with value: -1.288411906956877.
[I 2022-03-23 11:28:55,163] Trial 2952 finished with value: -1.288187985810191 and parameters: {'5*0+0': -0.10952460779946753, '5*0+1': 7.111695844277034, '5*0+2': 0.36543858030637966, '5*0+3': -4.892327873463161, '5*0+4': 2.485428555437557, '5*1+0': 0.15724390363918672, '5

[I 2022-03-23 11:29:27,455] Trial 2963 finished with value: -1.2881565452022299 and parameters: {'5*0+0': -0.1039567430895763, '5*0+1': 7.09152796295508, '5*0+2': 0.3695919493507698, '5*0+3': -4.842938052235647, '5*0+4': 2.463451898796226, '5*1+0': 0.1540791321375156, '5*1+1': 8.808286955287157, '5*1+2': 6.74361675413808, '5*1+3': -5.299094179282609, '5*1+4': 4.402940958421948, '5*2+0': 5.947925036408639, '5*2+1': 3.412173015960975, '5*2+2': 7.416318454923979, '5*2+3': 0.6616348579615602, '5*2+4': -5.673775287026921, '5*3+0': -0.12077320634074394, '5*3+1': 4.780933899129526, '5*3+2': 1.645452027440261, '5*3+3': -3.102206303029538, '5*3+4': 4.707320739271812}. Best is trial 2953 with value: -1.2884130126204272.
[I 2022-03-23 11:29:30,397] Trial 2964 finished with value: -1.288132067995749 and parameters: {'5*0+0': -0.10329236880311864, '5*0+1': 7.113660122609927, '5*0+2': 0.35414401730947176, '5*0+3': -4.800240386412654, '5*0+4': 2.4876885412331653, '5*1+0': 0.14504607929480656, '5*1+1'

[I 2022-03-23 11:30:03,122] Trial 2975 finished with value: -1.2880972114262335 and parameters: {'5*0+0': -0.10553683871904708, '5*0+1': 7.1386872406696265, '5*0+2': 0.3879030390217536, '5*0+3': -4.867790040280914, '5*0+4': 2.46462992121667, '5*1+0': 0.15421701000479407, '5*1+1': 8.793545076298061, '5*1+2': 6.770816600360154, '5*1+3': -5.276285303372546, '5*1+4': 4.3787308806711165, '5*2+0': 5.937180960281366, '5*2+1': 3.43023323018157, '5*2+2': 7.412491510833915, '5*2+3': 0.64500073655428, '5*2+4': -5.68714088215167, '5*3+0': -0.1258649947281977, '5*3+1': 4.746634153163424, '5*3+2': 1.5968506966519622, '5*3+3': -3.0886882292419116, '5*3+4': 4.672627405876251}. Best is trial 2953 with value: -1.2884130126204272.
[I 2022-03-23 11:30:06,155] Trial 2976 finished with value: -1.287995797674279 and parameters: {'5*0+0': -0.11630103220092851, '5*0+1': 7.149949765953685, '5*0+2': 0.37753030523408276, '5*0+3': -4.8208505068788075, '5*0+4': 2.475902247583423, '5*1+0': 0.16123589120431414, '5*1+

[I 2022-03-23 11:30:38,680] Trial 2987 finished with value: -1.2878882421706976 and parameters: {'5*0+0': -0.12133938865888641, '5*0+1': 7.171184233824938, '5*0+2': 0.4118024276925528, '5*0+3': -4.867622288794574, '5*0+4': 2.460411346078038, '5*1+0': 0.196824521887634, '5*1+1': 8.732379366051736, '5*1+2': 6.779902217794337, '5*1+3': -5.247976410405459, '5*1+4': 4.452901496822147, '5*2+0': 5.965645217643784, '5*2+1': 3.378614674425556, '5*2+2': 7.393663994093317, '5*2+3': 0.6236707109729316, '5*2+4': -5.704202360322681, '5*3+0': -0.12892180398055006, '5*3+1': 4.726745722319658, '5*3+2': 1.6345011016424598, '5*3+3': -3.08502730425625, '5*3+4': 4.695745941452546}. Best is trial 2953 with value: -1.2884130126204272.
[I 2022-03-23 11:30:41,684] Trial 2988 finished with value: -1.2883104677349488 and parameters: {'5*0+0': -0.11756876517010585, '5*0+1': 7.161497136818445, '5*0+2': 0.3822582347221538, '5*0+3': -4.885308222114342, '5*0+4': 2.4543621439967223, '5*1+0': 0.16420577850501983, '5*1+

[I 2022-03-23 11:31:13,963] Trial 2999 finished with value: -1.2881538941432604 and parameters: {'5*0+0': -0.09899150087259065, '5*0+1': 7.141488851168577, '5*0+2': 0.38538998323807283, '5*0+3': -4.874533292595835, '5*0+4': 2.4585583498012333, '5*1+0': 0.17289360114355162, '5*1+1': 8.76829035613581, '5*1+2': 6.765952022812544, '5*1+3': -5.300778532250003, '5*1+4': 4.373416690057736, '5*2+0': 5.970897403415042, '5*2+1': 3.3768235150609103, '5*2+2': 7.382847085921789, '5*2+3': 0.6741591384156678, '5*2+4': -5.655648980098849, '5*3+0': -0.11045818687929707, '5*3+1': 4.748437139834655, '5*3+2': 1.6779351148311434, '5*3+3': -3.150961572937194, '5*3+4': 4.678902764483229}. Best is trial 2990 with value: -1.2884621713479936.


In [21]:
theta = np.zeros(5*N)
i=0
for value in study.best_trial.params.items():
    theta[i] = value[1]
    i+=1

psi_var = psi_0
for i in range(N):

    psi_var = ms_gate_global(theta[5*i+0])@psi_var
    psi_var = rot_sigma_e(0,2,0,theta[5*i+2])@rot_sigma_e(0,1,0,theta[5*i+1])@psi_var
    psi_var = rot_sigma_o(0,2,0,theta[5*i+4])@rot_sigma_o(0,1,0,theta[5*i+3])@psi_var
    psi_var = rot_sigma(0,0,2,0,theta[5*i+2])@rot_sigma(0,0,1,0,theta[5*i+1])@psi_var
    psi_var = rot_sigma(L-1,0,2,0,theta[5*i+2])@rot_sigma(L-1,0,1,0,theta[5*i+1])@psi_var
    
print(np.abs(np.dot(np.conj(psi_var),eigenvec[:,0]))**2)

0.9200102247050369


In [22]:
string = ""
for i in range(basis.Ns):
    if np.abs(psi_var[i])>=1e-5:
        string+=str(psi_var[i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)

(1.4852451114803473e-06-1.2600130055581601e-05j)|2 2 2 2 2 2 1>
(1.7444171125834897e-06+1.0189429116819969e-05j)|2 2 2 2 2 2 0>
(-6.260276700337326e-05+0.0001288834218503023j)|2 2 2 2 2 1 2>
(0.00010365439273080734-0.00028242014018711097j)|2 2 2 2 2 1 1>
(7.66482621826357e-05+1.5040023783104333e-05j)|2 2 2 2 2 1 0>
(-7.3207268431013705e-06+1.3888854717038919e-05j)|2 2 2 2 2 0 2>
(1.4581790377517792e-06-3.0327770689659724e-05j)|2 2 2 2 2 0 1>
(1.4226009088054398e-05-9.766915035093932e-07j)|2 2 2 2 2 0 0>
(1.4852451229352474e-06-1.2600130037322825e-05j)|2 2 2 2 1 2 2>
(1.5592752912958744e-06+3.7296249473149424e-05j)|2 2 2 2 1 2 1>
(-7.946530908440801e-06-2.218811063195561e-05j)|2 2 2 2 1 2 0>
(0.00010365439273304263-0.00028242014016615453j)|2 2 2 2 1 1 2>
(-0.0001151618236456598+0.00060358397370249j)|2 2 2 2 1 1 1>
(-0.00016466796609639894+4.54205447808242e-06j)|2 2 2 2 1 1 0>
(1.4581790672194923e-06-3.032777071640611e-05j)|2 2 2 2 1 0 2>
(2.4219764890222584e-05+6.058597898420556e-05j)|2

In [23]:
string = ""
for i in range(basis.Ns):
    if np.abs(eigenvectors[:,0][i])>=1e-5:
        string+=str(eigenvectors[:,0][i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)

(-0.00022763676459291275-0.0003681681720939992j)|2 2 2 2 2 2 2>
(-0.0008323333931273763-0.0013461738680757712j)|2 2 2 2 2 1 2>
(0.001547680765217765+0.002503140471599908j)|2 2 2 2 2 1 1>
(-0.0006750766520319386-0.0010918347809283954j)|2 2 2 1 2 2 2>
(-0.0031530150978460736-0.005099526902946758j)|2 2 2 1 2 1 2>
(0.00612633914391514+0.009908430600903089j)|2 2 2 1 2 1 1>
(0.005971424157538211+0.009657878949014933j)|2 2 2 1 1 1 2>
(-0.013042574881883755-0.02109439994103347j)|2 2 2 1 1 1 1>
(0.0048905295977886756+0.007909694841361798j)|2 2 2 1 1 0 1>
(-0.0008323333931231757-0.0013461738682099622j)|2 1 2 2 2 2 2>
(-0.0031273291615768273-0.005057983770694168j)|2 1 2 2 2 1 2>
(0.005853659581371855+0.009467412489024605j)|2 1 2 2 2 1 1>
(-0.003153015097886453-0.00509952690298934j)|2 1 2 1 2 2 2>
(-0.015019182697014615-0.02429126529552277j)|2 1 2 1 2 1 2>
(0.029297257940189638+0.04738390093610306j)|2 1 2 1 2 1 1>
(0.029655438305636387+0.047963203715444866j)|2 1 2 1 1 1 2>
(-0.06489830760330569-0.